In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.12) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [2]:
from mlrl.experiments.train_maze_agent import (
    create_maze_meta_env, RestrictedActionsMazeState, create_batched_tf_meta_env, 
    create_agent, create_training_run, get_maze_name
)
from tf_agents.environments.tf_py_environment import TFPyEnvironment
from tf_agents.environments.gym_wrapper import GymWrapper
from tf_agents.environments.batched_py_environment import BatchedPyEnvironment
from tf_agents.train.utils import spec_utils

args = {'agent': 'ppo_agent'}
env_batch_size = 2

env = BatchedPyEnvironment([
    GymWrapper(create_maze_meta_env(RestrictedActionsMazeState, args)) 
    for _ in range(env_batch_size)
])

eval_env = BatchedPyEnvironment([
    GymWrapper(create_maze_meta_env(RestrictedActionsMazeState, args)) 
    for _ in range(env_batch_size)
])

env.reset()

observation_tensor_spec, action_tensor_spec, time_step_tensor_spec = (
      spec_utils.get_tensor_specs(env))

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: No such file or directory
ALSA lib confmisc.c:1246:(snd_func_refer) error evaluating name
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_refer returned error: No such file or directory
ALSA lib conf.c:5220:(snd_config_expand) Evaluate error: No such file or directory
ALSA lib pcm.c:2642:(snd_pcm_open_noupdate) Unknown PCM default
ALSA lib confmisc.c:767:(parse_card) cannot find card '0'
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_card_driver returned error: No such file or directory
ALSA lib confmisc.c:392:(snd_func_concat) error evaluating strings
ALSA lib conf.c:4732:(_snd_config_evaluate) function snd_func_concat returned error: N

In [3]:
from tf_agents.agents.ppo.ppo_agent import PPOAgent
from tf_agents.networks.sequential import Sequential
from tf_agents.networks.value_network import ValueNetwork
from tf_agents.networks.actor_distribution_network import ActorDistributionNetwork
from tf_agents.train.utils import train_utils

from mlrl.meta.search_networks import SearchActorNetwork, SearchValueNetwork
from mlrl.meta.search_networks import SearchTransformer, CategoricalNetwork, SearchActorLogitsNetwork


custom_objects = {
    'SearchActorLogitsNetwork': SearchActorLogitsNetwork,
    'SearchTransformer': SearchTransformer
}

with tf.keras.utils.custom_object_scope(custom_objects):
    actor_dist_net = ActorDistributionNetwork(
        observation_tensor_spec, action_tensor_spec,
        preprocessing_layers=SearchActorLogitsNetwork(),
        fc_layer_params=None,
        discrete_projection_net=lambda spec: CategoricalNetwork(spec)
    )

value_net = ValueNetwork(
    observation_tensor_spec,
    preprocessing_layers=SearchTransformer(3, 16, 2),
    preprocessing_combiner=tf.keras.layers.Lambda(lambda x: tf.reduce_sum(x[0], axis=-2)),
    batch_squash=True,
    fc_layer_params=None
)

train_step = train_utils.create_train_step()

agent = PPOAgent(
    time_step_tensor_spec,
    action_tensor_spec,
    actor_net=actor_dist_net,
    value_net=value_net,
    optimizer=tf.keras.optimizers.Adam(),
    train_step_counter=train_step,
    compute_value_and_advantage_in_train=False,
    update_normalizers_in_train=False,
    normalize_observations=False,
    discount_factor=0.99,
    num_epochs=1,  # deprecated param
)

2022-10-03 21:09:23.745991: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-03 21:09:23.812865: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-03 21:09:23.813293: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:961] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-10-03 21:09:23.814388: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate

In [4]:
from tf_agents.replay_buffers import tf_uniform_replay_buffer

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=env.batch_size or 1,
    max_length=10000
)


def preprocess_seq(experience, info):
    return agent.preprocess_sequence(experience), info


def dataset_fn():
    ds = replay_buffer.as_dataset(sample_batch_size=8, num_steps=32)
    return ds.map(preprocess_seq)

In [6]:
from tf_agents.train import actor
from tf_agents.train import learner
from tf_agents.metrics import py_metrics
from tf_agents.train import triggers

import os

root_dir = './runs/ppo_agent/test'

# summary_interval = 1000
# collect_sequence_length = 2000
# policy_save_interval = 5000
summary_interval = 1
collect_sequence_length = 256
policy_save_interval = 2

saved_model_dir = os.path.join(root_dir, learner.POLICY_SAVED_MODEL_DIR)
collect_env_step_metric = py_metrics.EnvironmentSteps()
learning_triggers = [
    triggers.PolicySavedModelTrigger(
        saved_model_dir,
        agent,
        train_step,
        interval=policy_save_interval,
        metadata_metrics={
            triggers.ENV_STEP_METADATA_KEY: collect_env_step_metric
        }),
    triggers.StepPerSecondLogTrigger(train_step, interval=summary_interval),
]

collect_actor = actor.Actor(
    env,
    agent.collect_policy,
    train_step,
    steps_per_run=collect_sequence_length,
    observers=[replay_buffer.add_batch],
    metrics=actor.collect_metrics(buffer_size=10) + [collect_env_step_metric],
    reference_metrics=[collect_env_step_metric],
    summary_dir=os.path.join(root_dir, learner.TRAIN_DIR),
    summary_interval=summary_interval)

collect_actor.run()

In [7]:
from tf_agents.policies import py_tf_eager_policy

eval_greedy_policy = py_tf_eager_policy.PyTFEagerPolicy(
    agent.policy, use_tf_function=True)

eval_steps=1000

eval_actor = actor.Actor(
    eval_env,
    eval_greedy_policy,
    train_step,
    metrics=actor.eval_metrics(eval_steps),
    reference_metrics=[collect_env_step_metric],
    summary_dir=os.path.join(root_dir, 'eval'),
    steps_per_run=eval_steps)

eval_actor.run_and_log()

In [9]:
from tf_agents.train.ppo_learner import PPOLearner

ppo_learner = PPOLearner(
    root_dir,
    train_step,
    agent,
    experience_dataset_fn=dataset_fn,
    normalization_dataset_fn=dataset_fn,
    num_samples=1, num_epochs=10,  # num samples * num epochs = num iterations per run call
    triggers=learning_triggers
)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [20]:
from mlrl.utils.render_utils import create_policy_eval_video, embed_mp4

embed_mp4(create_policy_eval_video(agent.policy, env, max_steps=300))

In [25]:
videos_dir = root_dir + '/videos'

from pathlib import Path
Path(videos_dir).mkdir(parents=True, exist_ok=True) 

In [28]:
eval_interval = 5


for i in range(100):
      print(f'Iteration: {i}')
      for metric in eval_actor.metrics:
            metric.reset()
      for metric in collect_actor.metrics:
            metric.reset()

      collect_actor.run()
      print('Collect stats:')
      print(', '.join([f'{metric.name}: {metric.result():.3f}' for metric in collect_actor.metrics]))

      loss_info = ppo_learner.run()
      print('Training info:')
      print(f'Loss: {loss_info.loss:.5f}, '
            f'KL Penalty Loss: {loss_info.extra.kl_penalty_loss:.5f}, '
            f'Entropy: {loss_info.extra.entropy_regularization_loss:.5f}, '
            f'Value Estimation Loss: {loss_info.extra.value_estimation_loss:.5f}, '
            f'PG Loss {loss_info.extra.policy_gradient_loss:.5f}')

      if i % eval_interval == 0:
            eval_actor.run_and_log()
            print('Evaluation stats:')
            print(', '.join([f'{metric.name}: {metric.result():.3f}' for metric in eval_actor.metrics]))
            create_policy_eval_video(agent.policy, env, max_steps=300, filename=f'{videos_dir}/video_{i}.mp4')
            print()

Iteration: 0


Exception: Node index out of bounds: node_idx=<tf.Tensor: shape=(), dtype=int64, numpy=7>, action=<tf.Tensor: shape=(), dtype=int64, numpy=0>, len(self.node_list)=1

In [55]:
from mlrl.utils.render_utils import create_policy_eval_video, embed_mp4

embed_mp4(create_policy_eval_video(agent.collect_policy, env, max_steps=30))